## NSC

In [ ]:
try:
    from bs4 import BeautifulSoup
    from selenium import webdriver

    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service

    import pandas as pd

    if input("Deseja rodar preparação de ambiente (RECOMENDADO PARA PRIMEIRA VEZ): [sim/não]") in ["sim", "Sim", "S", "s"]:
        raise Exception("Preparação de ambiente solicitada")
except:
    print("Configurando ambiente")
    
    import os
    import subprocess
    
    print("Checking for not installed packages...")
    
    result = subprocess.run(["pip", "list"], stdout=subprocess.PIPE, text=True)

    if not all([lib in result.stdout for lib in ["selenium","wget","pandas","openpyxl", "beautifulsoup4"]]):
        print("Installing packages...")
        os.system("pip install --upgrade selenium wget pandas openpyxl beautifulsoup4")
    
    print("All packages are installed!")
    
    
    print("Checking for outdated packages...")
    result = subprocess.run(["pip", "list", "--outdated"], stdout=subprocess.PIPE, text=True)
    
    if any([lib in result.stdout for lib in ["selenium","wget","pandas","openpyxl", "beautifulsoup4"]]):
        print("Updating packages...")
        os.system("pip install --upgrade selenium wget pandas openpyxl beautifulsoup4")

    print("All packages are updated!")
    
    import wget
    import zipfile
    
    if "chromedriver" not in os.listdir():
        print("Downloading chromedriver")
        filename = wget.download("https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/win64/chromedriver-win64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./chromedriver")
    else:
        print("Chromedriver found!")
    
    if "chrome" not in os.listdir():
        print("Downloading chrome")
        filename = wget.download("https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/win64/chrome-win64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./chrome")
    else:
        print("Chrome found!")


    from selenium import webdriver
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service

    import pandas as pd

options = webdriver.ChromeOptions()

options.binary_location = "./chrome/chrome-win64/chrome.exe"

driverpath = Service("./chromedriver/chromedriver-win64/chromedriver.exe")

options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')  # Evita problemas de memória compartilhada
options.add_argument('--disable-web-security')
options.add_argument('--disable-site-isolation-trials')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument('--disable-notifications')

options.page_load_strategy = 'eager'

driver = webdriver.Chrome(service=driverpath, options=options)

def articleFormatter(article, tag): 
    return {
        "title": article.find("h3").text.strip(),
        "link": article.find("a").get_attribute_list("href")[0],
        "data": article.find("div", class_="date").text.strip(),
        "tag": tag
    }

def SCfilter(article):
    cidades_sc1 = pd.read_excel('./planilhas/cidade_sc1.xlsx')
    
    for key in [" sc ", "santa catarina", " sc", "sc "]:
        if key in article["title"].lower():
            return True
        
        
    for key in ["-sc-", "santa-catarina", "-sc", "sc-"]:
        if key in article["link"].split("/")[-1].lower():
            return True

    for cidade in cidades_sc1["MUNICIPIO"]:
        if cidade.lower() in article["title"].lower() or cidade.lower() in article["link"].split("/")[-1].lower():
            return True

    return False

def getNewsByTags(tags):
    global driver
    allNews = []

    pageCounter = 0
    for tag in tags.keys():
        for page in range(tags[tag]):
            if pageCounter % 10 == 0:
                print(f"\nPlanilha salva com {len(allNews)} notícias para backup...")
                storeAsExcel(allNews)
                print("Salvo\n")
            
            acessed = False
            
            while not acessed:
                try:
                    driver.get(f"https://www.nsctotal.com.br/tag/{tag}?page={page}")
                    acessed = True
                except:
                    print("Erro ao acessar a página, reiniciando navegador...")
                    driver.quit()
                    driver = webdriver.Chrome(service=driverpath, options=options)
    
            try:
                WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, "date") ) )
                driver.implicitly_wait(5)
            except:
                continue            
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            news = soup.find_all('div', class_='featured-news-thumb')
            
            parsedNews = [articleFormatter(article, tag) for article in news]

            parsedNews = list(filter(SCfilter, parsedNews))
            
            allNews += parsedNews
            print(f"\nNoticias coletadas: {len(parsedNews)}\nTag: {tag}\nPágina:{page}\nTotal: {len(allNews)}\n")

            pageCounter += 1
        
        
            
    return allNews

def storeAsExcel(data):
    rows = list(map(lambda article: article.values(), data))
    df = pd.DataFrame(rows, columns=["title", "link", "data", "tag"])
    
    print(f"Número de noticias com duplicados: {len(df)}")
    
    df = df.drop("tag", axis=1)
    df = df.drop_duplicates()
    
    print(f"Número de noticias sem duplicados: {len(df)}")
    
    df.to_excel("./planilhas/noticias.xlsx", index=False)
    

searchReference = {
    "chuvas": 52,
    "chuva em sc": 35,
    "chuvas em sc": 60,
    "chuva": 395,
    "chuva forte": 5,
    "chuvarada": 11,
    "temporal": 109,
    "tempestade": 21,
    "ciclone": 32,
    "ciclone bomba": 5,
    "ciclone extratropical": 6,
    "previsao do tempo": 708,
    "frente fria": 9,
    "enchente": 92,
    "enchentes": 22,
    "alagamento": 61,
    "alagamentos": 20,
    "deslizamento": 36,
    "deslizamentos": 9,
    "deslizamento de terra": 5
}

searchReference = {tag.replace(" ", "-"): searchReference[tag] for tag in searchReference}

data = getNewsByTags(searchReference)

print(data)

storeAsExcel(data)

## NDMAIS

In [ ]:
from httpx import TimeoutException
from tqdm import tqdm
try:
    from bs4 import BeautifulSoup
    from selenium import webdriver

    from selenium.webdriver.common.by import By
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service
    from selenium.webdriver.common.action_chains import ActionChains

    import pandas as pd
    import time


    if input("Deseja rodar preparação de ambiente (RECOMENDADO PARA PRIMEIRA VEZ): [sim/não]") in ["sim", "Sim", "S", "s"]:
        raise Exception("Preparação de ambiente solicitada")
except:
    print("Configurando ambiente")
    
    import os
    import subprocess
    
    print("Checking for not installed packages...")
    
    result = subprocess.run(["pip", "list"], stdout=subprocess.PIPE, text=True)

    if not all([lib in result.stdout for lib in ["selenium","wget","pandas","openpyxl", "beautifulsoup4"]]):
        print("Installing packages...")
        os.system("pip install --upgrade selenium wget pandas openpyxl beautifulsoup4")
    
    print("All packages are installed!")
    
    
    print("Checking for outdated packages...")
    result = subprocess.run(["pip", "list", "--outdated"], stdout=subprocess.PIPE, text=True)
    
    if any([lib in result.stdout for lib in ["selenium","wget","pandas","openpyxl", "beautifulsoup4"]]):
        print("Updating packages...")
        os.system("pip install --upgrade selenium wget pandas openpyxl beautifulsoup4")

    print("All packages are updated!")
    
    import wget
    import zipfile
    
    if "chromedriver" not in os.listdir():
        print("Downloading chromedriver")
        filename = wget.download("https://storage.googleapis.com/chrome-for-testing-public/134.0.6998.165/win64/chromedriver-win64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./chromedriver")
    else:
        print("Chromedriver found!")
    
    if "brave" not in os.listdir():
        print("Downloading brave")
        filename = wget.download("https://github.com/brave/brave-browser/releases/download/v1.76.82/brave-v1.76.82-win32-x64.zip")
        with zipfile.ZipFile(f"./{filename}", 'r') as zip_ref:
            zip_ref.extractall("./brave")
    else:
        print("Brave found!")


    from selenium import webdriver
    from selenium.webdriver.support.wait import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.webdriver.chrome.service import Service

    import pandas as pd

options = webdriver.ChromeOptions()

options.binary_location = "./brave/brave.exe"

driverpath = Service("./chromedriver/chromedriver-win64/chromedriver.exe")

# options.add_argument('--headless=new')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')  # Evita problemas de memória compartilhada
options.add_argument('--disable-web-security')
options.add_argument('--disable-site-isolation-trials')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--allow-running-insecure-content')
options.add_argument('--disable-notifications')

options.page_load_strategy = 'eager'

driver = webdriver.Chrome(service=driverpath, options=options)

def articleFormatter(article, tag, progress_bar=None): 
    progress_bar.update(1)
    return {
        "title": article.find("a").get_attribute_list("title")[0].strip(),
        "link": article.find("a").get_attribute_list("href")[0],
        "data": article.find("time").get_attribute_list("title")[0].strip(),
        "tag": tag
    }

def SCfilter(article):
    cidades_sc1 = pd.read_excel('./planilhas/cidade_sc1.xlsx')
    
    for key in [" sc ", "santa catarina", " sc", "sc "]:
        if key in article["title"].lower():
            return True
        
        
    for key in ["-sc-", "santa-catarina", "-sc", "sc-"]:
        if key in article["link"].split("/")[-1].lower():
            return True

    for cidade in cidades_sc1["MUNICIPIO"]:
        if cidade.lower() in article["title"].lower() or cidade.lower() in article["link"].split("/")[-1].lower():
            return True

    return False

def getNewsByTags(tags):
    global driver
    allNews = []

    for tag in tags:
        
        acessed = False
        
        while not acessed:
            try:
                driver.get(f"https://ndmais.com.br/?s={tag}")
                acessed = True
            except:
                print("Erro ao acessar a página, reiniciando navegador...")
                driver.quit()
                driver = webdriver.Chrome(service=driverpath, options=options)

        WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CLASS_NAME, "title-text")))


        while True:
            try:
                ActionChains(driver).scroll_by_amount(0, 10000).perform()
                
                time.sleep(1)
                
                WebDriverWait(driver, 10).until( EC.presence_of_element_located( (By.CSS_SELECTOR, "a[title=\"Veja Mais\"]") ) )
                                
                driver.find_element(By.CSS_SELECTOR, "a[title=\"Veja Mais\"]").click()

                WebDriverWait(driver, 10).until_not( EC.visibility_of_element_located( (By.CSS_SELECTOR, "i[class=\"button-icon fas fa-spin fa-sync nd-fa-loaded\"]") ) )

            except Exception as e:
                print(e)
                print(f"Page {tag} carregada completamente")
                break            

        ###CONCERTAR ESSA PARTE O ELEMENTO AS VEZES NÃO É CLICAVEL OU ENCONTADO
        
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        news = soup.find_all('div', class_='site-card-content')
        
        # print(f"Formatando notícias com a tag {tag}...")
        
        progress_bar = tqdm(total=len(news), desc=f"Formatando notícias com a tag {tag}", unit="notícias")
        parsedNews = [articleFormatter(article, tag, progress_bar) for article in news]

        print("Notícias formatadas!")

        print(f"Filtrando notícias com a tag {tag}...")
        parsedNews = list(filter(SCfilter, parsedNews))
                
        print("Notícias Filtradas!")
        
        allNews += parsedNews


        print(f"\nPlanilha salva com {len(allNews)} notícias para backup...")
        storeAsExcel(allNews)
        print("Salvo\n")
            
        print(f"\nNoticias coletadas: {len(parsedNews)}\nTag: {tag}\nTotal: {len(allNews)}\n")
        
            
    return allNews

def storeAsExcel(data):
    rows = list(map(lambda article: article.values(), data))
    df = pd.DataFrame(rows, columns=["title", "link", "data", "tag"])
    
    print(f"Número de noticias com duplicados: {len(df)}")
    
    df = df.drop("tag", axis=1)
    df = df.drop_duplicates()
    
    print(f"Número de noticias sem duplicados: {len(df)}")
    
    df.to_excel("./planilhas/noticias.xlsx", index=False)
    

searchReference = {
    "chuvas",
    "chuva em sc",
    "chuvas em sc",
    "chuva",
    "chuva forte",
    "chuvarada",
    "temporal",
    "tempestade",
    "ciclone",
    "ciclone bomba",
    "ciclone extratropical",
    "previsão do tempo",
    "frente fria",
    "enchente",
    "enchentes",
    "alagamento",
    "alagamentos",
    "deslizamento",
    "deslizamentos",
    "deslizamento de terra"
}

searchReference = list(map(lambda x: x.replace(" ", "+"), searchReference))

data = getNewsByTags(searchReference)

print(data)

storeAsExcel(data)

## Classificador de notícias

In [ ]:
try:
    import os
    import math
    import pandas as pd
    import json
    from dotenv import load_dotenv
    from google import genai
except:
    print("Configurando ambiente")
    
    os.system("pip install --upgrade google-genai python-dotenv")

    from dotenv import load_dotenv
    from google import genai
    
load_dotenv()

client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

def generateContent(prompt):
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,   
    )
    return response.text

def classifyTitles(data):
    return generateContent("Vou te passar titulos de noticias e quero que você classifique simplesmente como \"previsão\", para coisas que ainda não aconteceram, \"aconteceu\" para coisas que já aconteceram e \"historico\" para coisas que já aconteceram e não vão mais acontecer. Não quero explicações nem que mude os titulos, apenas a classificação. Exemplo: \"Amanhã vai chover em SC\" = previsão. Exemplo 2: \"Ontem choveu em SC\" = aconteceu. Exemplo 3: \"Em 2000 choveu em SC\" = historico. A formatação devera ser a seguinte <Titulo da notícia exatamente como foi enviado> = <Classificação>\n Titulos: " + "\n".join(data))

def dispatchTitles(articles, groupSize=300):
    titleClassification = []

    for i in range(0, math.ceil(len(articles)/groupSize)):
        start = i * groupSize
        end = (i + 1) * groupSize
        
        payload = classifyTitles([str(pos+start)+"."+article for pos,article in enumerate(articles[start:end])]).split("\n")[:-1]
        payload = [{x.split(" = ")[0].replace("*","").strip() : x.split(" = ")[1]} for x in payload]
        print(payload)
        print(len(payload))
        titleClassification += payload
        
        print(f"{min(end,len(articles))} de {len(articles)} classificações concluídas")

    return titleClassification

def dispatchRemainingTitles(articles, indexes, groupSize=200):
    titleClassification = []

    for i in range(0, math.ceil(len(indexes)/groupSize)):
        start = i * groupSize
        end = (i + 1) * groupSize
        
        selectedArticles = [articles[pos] for pos in indexes[start:end]]
        
        payload = classifyTitles([str(indexes[pos+start])+"."+article for pos,article in enumerate(selectedArticles)]).split("\n")[:-1]
        payload = [{x.split(" = ")[0].replace("*","").strip() : x.split(" = ")[1]} for x in payload]
        print(payload)
        print(len(payload))
        titleClassification += payload
        
        print(f"{min(end,len(indexes))} de {len(indexes)} classificações concluídas")

    print(titleClassification)
    return titleClassification


arquivoDeNoticias = input("Qual arquivo você quer classificar: ")
articles = pd.read_excel(f'./planilhas/{arquivoDeNoticias}')

#cleaning tht titles
articles["title"] = articles["title"].apply(lambda x: x.replace(" *", "").replace("*", "").replace("* ", "").replace("  ", " ").replace("  ", " ").replace("  ", " ").replace(u'\xa0', u' ').strip())

#Crate lookup table for title and index
lookupArticleIndex = {x:[] for x in list(articles["title"])}
[lookupArticleIndex[x].append(i) for i, x in enumerate(list(articles["title"]))]

print(lookupArticleIndex)

useBackup = (True if input("Você quer usar o backup de titulos? [sim/não]") == "sim" else False) if os.path.exists("backup_titles.json") else False

if (useBackup):
    print("Lendo backup de titulos...")
    with open("backup_titles.json", "r", encoding="utf-16") as arq:
        classifiedTitles = json.loads(arq.read())
else:
    classifiedTitles = dispatchTitles(list(articles["title"]))

    with open("backup_titles.json", "w", encoding="utf-16") as arq:
        arq.write(json.dumps(classifiedTitles))

coverage = set(map(lambda x: int(list(x.keys())[0].split(".")[0]) if "." in list(x.keys())[0][:5] else -1, classifiedTitles))
print("Not covered indexes:")
notCovered = list(set(range(len(list(articles["title"])))).difference(coverage))
print(notCovered)

while len(notCovered) > 0:
    classifiedTitles += dispatchRemainingTitles(list(articles["title"]), notCovered)
    
    print(classifiedTitles[-668:])
    
    coverage = set(map(lambda x: int(list(x.keys())[0].split(".")[0]) if "." in list(x.keys())[0][:7] else -1, classifiedTitles))
    print("Not covered indexes:")
    notCovered = list(set(range(len(list(articles["title"])))).difference(coverage))
    print(notCovered)


classifiedNews = []

for title in classifiedTitles:
    idx = int(list(title.keys())[0].split(".")[0])
    
    classifiedNews.append({
        "title": articles["title"][idx],
        "link": articles["link"][idx],
        "data": articles["data"][idx],
        "classificacao": list(title.values())[0]
    })

print("Salvando em planilha...")
df = pd.DataFrame(classifiedNews, columns=classifiedNews[0].keys())
df.to_excel("./planilhas/noticias_classificadas.xlsx", index=False)
print("Salvo com sucesso!")

In [ ]:
import pandas as pd

df = pd.read_excel("./planilhas/noticias.xlsx")

titles = list(df["title"])

lookupArticleIndex = {x:[] for x in list(articles["title"])}
[lookupArticleIndex[x].append(i) for i, x in enumerate(list(articles["title"]))]

print(articles["title"][6310])

# print(len(lookupArticleIndex.keys()))

# print([x for x in titles if "ondas de até 3 metros " in x])

In [ ]:
import json

print(len(json.loads(open("backup_titles.json", "r", encoding="utf-16").read())))